In [11]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from autoxgb import AutoXGB
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head()

,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
0,0,212990.0,0.642286,0.856520,0.707073,0.002001,10.0,NaN,-5.619088,0,0.082570,158.386236,4,0.734642,0
1,1,NaN,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1,0.127358,102.752988,3,0.711531,1
2,2,193213.0,NaN,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0,0.052282,178.685791,3,0.425536,0
3,3,249893.0,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0,0.035618,128.715630,3,0.453597,0
4,4,165969.0,0.493017,NaN,0.740982,0.002033,10.0,0.094891,-2.684095,0,0.050746,121.928157,4,0.741311,0


In [4]:
X = train.drop(["id", "song_popularity"], axis=1)
y = train.song_popularity

In [5]:
test_df = test.drop("id", 1)

In [6]:
# col_after transform
col_x = X.columns.to_list()

In [7]:
imp = IterativeImputer(max_iter=10)
X = imp.fit_transform(X)
x_test = imp.fit_transform(test_df)

In [8]:
X

array([[2.12990000e+05, 6.42285517e-01, 8.56520289e-01, ...,
        1.58386236e+02, 4.00000000e+00, 7.34641557e-01],
       [1.97014838e+05, 5.48656388e-02, 7.33288830e-01, ...,
        1.02752988e+02, 3.00000000e+00, 7.11530551e-01],
       [1.93213000e+05, 1.59724777e-01, 1.88387099e-01, ...,
        1.78685791e+02, 3.00000000e+00, 4.25536016e-01],
       ...,
       [1.60879000e+05, 8.77431248e-01, 4.09064592e-01, ...,
        9.95560744e+01, 3.00000000e+00, 1.77947389e-01],
       [1.93918000e+05, 1.73402378e-01, 3.65738206e-01, ...,
        1.39857384e+02, 3.00000000e+00, 7.72978406e-01],
       [1.96475000e+05, 7.11618441e-03, 3.54585290e-01, ...,
        1.01974949e+02, 3.00000000e+00, 5.88549127e-01]])

In [9]:
X = pd.DataFrame(columns=col_x, data=X)
X.head()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,212990.000000,0.642286,0.856520,0.707073,0.002001,10.0,0.215368,-5.619088,0.0,0.082570,158.386236,4.0,0.734642
1,197014.838414,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1.0,0.127358,102.752988,3.0,0.711531
2,193213.000000,0.159725,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0.0,0.052282,178.685791,3.0,0.425536
3,249893.000000,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0.0,0.035618,128.715630,3.0,0.453597
4,165969.000000,0.493017,0.644204,0.740982,0.002033,10.0,0.094891,-2.684095,0.0,0.050746,121.928157,4.0,0.741311


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

In [16]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [17]:
pred = rf.predict(X_test)
f1_score(y_test, pred)

0.13317619328226282

In [18]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.64      0.94      0.77      2557
           1       0.44      0.08      0.13      1443

    accuracy                           0.63      4000
   macro avg       0.54      0.51      0.45      4000
weighted avg       0.57      0.63      0.54      4000



In [19]:
from sklearn.metrics import roc_auc_score

In [20]:
roc_auc_score(y_test, pred)

0.5115831664521535